# Dicom Image Segmentation

In [1]:
import os
import random
import shutil
from time import time
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import pydicom
from tqdm import trange

## 1、Build Dicom_Seg Class

In [4]:
class Dicom_Seg():
    '''读取LDCT和NDCT图像，分割为matrix*matrix大小，并保存在指定路径。'''
    
    def __init__(self, LDCT_root, NDCT_root, LDCT_save_path, NDCT_save_path, matrix = 64):
        '''请指定分割的目标大小、LDCT和NDCT读取路径和保存路径'''
        super().__init__()
        self.LDCT_root = LDCT_root
        self.NDCT_root = NDCT_root
        self.LDCT_save_path = LDCT_save_path
        self.NDCT_save_path = NDCT_save_path
        self.matrix = matrix
        
        LDCT_list = os.listdir(LDCT_root)
        NDCT_list = os.listdir(NDCT_root)
        self.data_path = list(zip(LDCT_list, NDCT_list))
        
        if len(LDCT_list) == len(NDCT_list):
            self.len = len(LDCT_list)
        else:
            print('LDCT和NDCT图像数量不一致，请检查!')


    def autoprocess(self):
        '''调用该方法全自动处理'''
        start_time = time()
        
        print('将Dicom图像分割为：{0}×{0} matrix'.format(self.matrix))
        print('LDCT保存路径: ' + self.LDCT_save_path, 'NDCT保存路径: ' + self.NDCT_save_path, sep = '\n')
        print('开始分割。。。')
        
        for idx in trange(self.len):
            LD_name, ND_name = self.data_path[idx]
            LD_path = self.LDCT_root + '\\' + LD_name
            ND_path = self.NDCT_root + '\\' + ND_name
            self.process(idx, LD_path, ND_path, self.LDCT_save_path, self.NDCT_save_path, LD_name, ND_name)
            
        print('分割完成！！！\n')
        end_time = time()
        time_start = datetime.fromtimestamp(start_time)
        time_end = datetime.fromtimestamp(end_time)
        print('Start Time:   ' + time_start.strftime('%H:%M:%S.%f'))
        print('End Time:     ' + time_end.strftime('%H:%M:%S.%f'))
        print('Elapsed Time: ' + '{:.3f}s'.format(end_time - start_time))
  
    def process(self, idx, LD_path, ND_path, LDCT_save_path, NDCT_save_path, LD_name, ND_name):
        '''读取图像、分割、保存'''
        # 读取
        LD_ds, LD_image = self.get_dcm_array(LD_path)
        ND_ds, ND_image = self.get_dcm_array(ND_path)

        LD_image = LD_image[32:-32, 32:-32]
        ND_image = ND_image[32:-32, 32:-32]
        
        # 分割
        LD_seg_images = self.segment(self.matrix, LD_image)
        ND_seg_images = self.segment(self.matrix, ND_image)
        
        # 保存
        self.save(idx, LDCT_save_path, LD_seg_images, LD_ds, LD_name)
        self.save(idx, NDCT_save_path, ND_seg_images, ND_ds, ND_name)
    
    @staticmethod
    def get_dcm_array(path):
        '''读取dcm pixel_array'''
        ds = pydicom.filereader.dcmread(path)
        return ds, ds.pixel_array
    
    @staticmethod
    def segment(matrix, image):
        '''按照matrix大小进行分割'''
        n = image.shape[0]/matrix
        n = int(n)
        images = []
        for y in range(n):
            for x in range(n):
                images.append(image[y*matrix:(y+1)*matrix, x*matrix:(x+1)*matrix])
        return images
    
    @staticmethod
    def save(idx, save_path, seg_images, ds, name):
        for i in range(len(seg_images)):
            ds.PixelData = seg_images[i].tobytes()
            ds.Rows, ds.Columns = seg_images[i].shape
            ds.save_as(save_path + "\\" + "{}-{}.dcm".format(name, i+1))

## 2、Instantiate Dicom_Seg()

In [6]:
LDCT_root = r'E:\NBIA\Sampling\Dataset\LDCT5'
NDCT_root = r'E:\NBIA\Sampling\Dataset\NDCT5'
LDCT_save_path = r'E:\NBIA\Sampling\Dataset\LDCT5set'
NDCT_save_path = r'E:\NBIA\Sampling\Dataset\NDCT5set'
if not os.path.isdir(NDCT_save_path):
    os.mkdir(NDCT_save_path)
if not os.path.isdir(LDCT_save_path):
    os.mkdir(LDCT_save_path)
matrix = 64
seg = Dicom_Seg(LDCT_root, NDCT_root, LDCT_save_path, NDCT_save_path, matrix = matrix)

## 3、Process

In [7]:
seg.autoprocess()

  0%|          | 0/260 [00:00<?, ?it/s]将Dicom图像分割为：64×64 matrix
LDCT保存路径: E:\NBIA\Sampling\Dataset\LDCT5set
NDCT保存路径: E:\NBIA\Sampling\Dataset\NDCT5set
开始分割。。。
100%|██████████| 260/260 [03:53<00:00,  1.11it/s]分割完成！！！

Start Time:   20:35:17.356852
End Time:     20:39:10.788025
Elapsed Time: 233.431s



# Image Sampling

In [6]:
count = 0
sep = 5  #抽样间隔

root = r'E:\NBIA\Sampling\Dataset\train'
NDCT_save_path = r'E:\NBIA\Sampling\Dataset\NDCT5'
LDCT_save_path = r'E:\NBIA\Sampling\Dataset\LDCT5'

if not os.path.isdir(NDCT_save_path):
    os.mkdir(NDCT_save_path)
if not os.path.isdir(LDCT_save_path):
    os.mkdir(LDCT_save_path)
    
NDCT_lable_str = 'Full dose images'
LDCT_lable_str = 'Low Dose Images'
patient_list = os.listdir(root)
print('一共{}例，每隔{}张图像分层抽样'.format(len(patient_list), sep))
for patient in patient_list:
    patient_root = root + '\\' + patient + '\\' + os.listdir(root + '\\' + patient)[0]
    paths = os.listdir(patient_root)
    if NDCT_lable_str in paths[0]:
        ND_path = patient_root + '\\' + paths[0]
        LD_path = patient_root + '\\' + paths[1]
    elif NDCT_lable_str in paths[1]:
        ND_path = patient_root + '\\' + paths[1]
        LD_path = patient_root + '\\' + paths[0]
        
    print(ND_path, LD_path, sep = '\n')
    ND_list = os.listdir(ND_path)
    LD_list = os.listdir(LD_path)
    
    idx = random.randint(0, sep-1)
    dcm_num = len(os.listdir(ND_path))//sep
    if len(os.listdir(ND_path))%sep >= (idx+1):
        dcm_num += 1
        
    print(patient)
    for i in trange(dcm_num):
        if ND_list[idx] != LD_list[idx]:
            break
        ND_new_name = 'NDCT-' + patient +'_'+ ND_list[idx]
        LD_new_name = 'LDCT-' + patient +'_'+ LD_list[idx]
        shutil.copy(os.path.join(ND_path, ND_list[idx]),os.path.join(NDCT_save_path, ND_new_name)) 
        shutil.copy(os.path.join(LD_path, LD_list[idx]),os.path.join(LDCT_save_path, LD_new_name)) 
        
        idx += sep
        count += 1
print('一共{}组图像。'.format(count))

 10%|█         | 2/20 [00:00<00:01, 16.44it/s]一共10例，每隔5张图像分层抽样
E:\NBIA\Sampling\Dataset\train\L004\08-21-2018-84608\1.000000-Full dose images-59704
E:\NBIA\Sampling\Dataset\train\L004\08-21-2018-84608\1.000000-Low Dose Images-35583
L004
 21%|██▏       | 6/28 [00:00<00:00, 49.28it/s]E:\NBIA\Sampling\Dataset\train\L072\08-26-2018-21862\1.000000-Full dose images-12733
E:\NBIA\Sampling\Dataset\train\L072\08-26-2018-21862\1.000000-Low Dose Images-71203
L072
 12%|█▏        | 3/26 [00:00<00:00, 29.49it/s]E:\NBIA\Sampling\Dataset\train\L081\08-22-2018-26950\1.000000-Full dose images-01818
E:\NBIA\Sampling\Dataset\train\L081\08-22-2018-26950\1.000000-Low Dose Images-38896
L081
 23%|██▎       | 6/26 [00:00<00:00, 50.53it/s]E:\NBIA\Sampling\Dataset\train\L110\08-23-2018-87457\1.000000-Full dose images-50611
E:\NBIA\Sampling\Dataset\train\L110\08-23-2018-87457\1.000000-Low Dose Images-35324
L110
 23%|██▎       | 7/31 [00:00<00:00, 41.53it/s]E:\NBIA\Sampling\Dataset\train\L114\08-24-2018-12249\1.00

In [3]:
count = 0
sep = 40  #抽样间隔

root = r'E:\NBIA\LDCT-and-Projection-data'
Noise_save_path = r'E:\NBIA\Sampling\Noise_blue'
if not os.path.isdir(Noise_save_path):
    os.mkdir(Noise_save_path)

NDCT_lable_str = 'Full dose images'
LDCT_lable_str = 'Low Dose Images'
patient_list = os.listdir(root)
print('一共{}例，每隔{}张图像分层抽样'.format(len(patient_list), sep))
for patient in patient_list:
    patient_root = root + '\\' + patient + '\\' + os.listdir(root + '\\' + patient)[0]
    paths = os.listdir(patient_root)
    if NDCT_lable_str in paths[0]:
        ND_path = patient_root + '\\' + paths[0]
        LD_path = patient_root + '\\' + paths[1]
    elif NDCT_lable_str in paths[1]:
        ND_path = patient_root + '\\' + paths[1]
        LD_path = patient_root + '\\' + paths[0]
        
#     print(ND_path, LD_path, sep = '\n')
    ND_list = os.listdir(ND_path)
    LD_list = os.listdir(LD_path)
    
    idx = random.randint(0, 9)
    dcm_num = len(os.listdir(ND_path))//sep
    if len(os.listdir(ND_path))%10 >= (idx+1):
        dcm_num += 1
        
    print(patient)
    for i in trange(dcm_num):
        if ND_list[idx] != LD_list[idx]:
            break
        Noise_new_name = 'Noise-' + patient +'_'+ ND_list[idx]
        nd = os.path.join(ND_path, ND_list[idx])
        ld = os.path.join(LD_path, LD_list[idx])  
        Noise = os.path.join(Noise_save_path, Noise_new_name)
        
        nd_ds = pydicom.filereader.dcmread(nd)
        ld_ds = pydicom.filereader.dcmread(ld)
        ld_img = ld_ds.pixel_array.astype(np.int16)
        nd_img = nd_ds.pixel_array.astype(np.int16)
        noise_img = ld_img - nd_img
        
        plt.imsave('{}.jpg'.format(Noise), noise_img[32:-32, 32:-32], cmap = plt.cm.Greys_r) 
        
        idx += 10
        count += 1
print('一共{}组图像。'.format(count))

  0%|          | 0/3 [00:00<?, ?it/s]一共50例，每隔40张图像分层抽样
L004
100%|██████████| 4/4 [00:00<00:00,  5.38it/s]一共182组图像。



In [4]:
count = 0
sep = 1
if not os.path.isdir(r'E:\NBIA\Sampling\LDCT_ALL'):
    os.mkdir(r'E:\NBIA\Sampling\LDCT_ALL')
    
root = r'E:\NBIA\LDCT-and-Projection-data'
LDCT_save_path = r'E:\NBIA\Sampling\LDCT_ALL'

LDCT_lable_str = 'Low Dose Images'
patient_list = os.listdir(root)
print('一共{}例，每隔{}张图像分层抽样'.format(len(patient_list), sep))
for patient in patient_list:
    patient_root = root + '\\' + patient + '\\' + os.listdir(root + '\\' + patient)[0]
    paths = os.listdir(patient_root)
    if LDCT_lable_str in paths[0]:
        LD_path = patient_root + '\\' + paths[1]
    elif LDCT_lable_str in paths[1]:
        LD_path = patient_root + '\\' + paths[0]
        
    LD_list = os.listdir(LD_path)
    dcm_num = len(LD_list)
        
    print(patient)
    for i in trange(dcm_num):
        LD_new_name = 'LDCT-' + patient +'_'+ LD_list[i]
        shutil.copy(os.path.join(LD_path, LD_list[i]),os.path.join(LDCT_save_path, LD_new_name)) 
        count += 1
print('一共{}组图像。'.format(count))

  2%|███▏                                                                                                                                                    | 2/97 [00:00<00:09,  9.80it/s]

一共50例，每隔1张图像分层抽样
L004


  1%|██                                                                                                                                                     | 2/144 [00:00<00:07, 18.46it/s]

L006


  1%|█▉                                                                                                                                                     | 2/153 [00:00<00:10, 14.29it/s]

...

L277


  1%|█▉                                                                                                                                                     | 2/154 [00:00<00:08, 17.76it/s]

L299


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 154/154 [00:05<00:00, 25.96it/s]

一共7380组图像。
